In [3]:
#!pip install -U langchain langchain-openai

In [ ]:
import os
import json
from datetime import datetime
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, ToolMessage
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client via LangChain
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    temperature=0
)

# History file path
HISTORY_FILE = "tlof_history.json"

# Function to load history
def load_history():
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, 'r') as f:
            return json.load(f)
    return []

# Function to save history
def save_history(entry):
    history = load_history()
    history.append(entry)
    with open(HISTORY_FILE, 'w') as f:
        json.dump(history, f, indent=2)

# Function to get the last configuration
def get_last_config():
    history = load_history()
    if history:
        return history[-1]['output']
    return None

# Define tool schema for TLOF layout generation
@tool
def generate_landing_surface_layout(
    aircraft: str = None,
    diameter: float = 30.0,
    shape_type: str = "Rectangle",
    position: list = None,
    sides: int = 4,
    width: float = 30.0,
    length: float = 30.0,
    height: float = 0.01,
    rotation: float = 0.0,
    transparency: float = 1.0,
    elevation: float = 0.0,
    markingType: str = "solid",
    markingColor: str = "white",
    markingThickness: float = 0.5,
    dashLength: float = 1.0,
    dashDistance: float = 1.0,
    landingMarker: str = None,
    markerScale: float = 1.0,
    markerThickness: float = 0.02,
    markerRotation: float = 0.0,
    markerColor: str = "white",
    letterThickness: float = 0.5,
    tdpcCategory: bool = False,
    tdpcType: str = "circle",
    tdpcScale: float = 5.0,
    tdpcThickness: float = 0.5,
    tdpcExtrusion: float = 0.02,
    tdpcRotation: float = 0.0,
    tdpcColor: str = "white",
    lightColor: str = None,
    lightScale: float = 1.0,
    lightDistance: float = 1.0,
    lightRadius: float = 0.3,
    lightHeight: float = 0.2,
    safetyAreaType: str = "offset",
    dValue: float = 10.0,
    multiplier: float = 1.5,
    offsetDistance: float = 3.0,
    curveAngle: float = 45.0,
    netHeight: float = 15.0,
    safetyNetTransparency: float = 0.5
) -> dict:
    """Generate a structured JSON layout for a TLOF (Touchdown and Lift-Off Area)."""
    return {
        "aircraft": aircraft,
        "diameter": max(1.0, min(diameter, 200.0)),
        "shape_type": shape_type if shape_type in ["Circle", "Polygon", "Rectangle"] else "Rectangle",
        "position": position if position and len(position) == 2 else [0, 0],
        "sides": max(3, min(sides, 70)),
        "width": max(30.0, min(width, 30.0)),
        "length": max(30.0, min(length, 30.0)),
        "height": max(0.01, min(height, 5.0)),
        "rotation": max(0.0, min(rotation, 359.0)),
        "transparency": max(0.0, min(transparency, 1.0)),
        "elevation": max(0.0, min(elevation, 1000.0)),
        "markingType": markingType if markingType in ["solid", "dashed"] else "solid",
        "markingColor": markingColor if markingColor in ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"] else "white",
        "markingThickness": max(0.1, min(markingThickness, 1.5)),
        "dashLength": max(0.5, min(dashLength, 5.0)),
        "dashDistance": max(0.5, min(dashDistance, 5.0)),
        "landingMarker": landingMarker,
        "markerScale": max(0.5, min(markerScale, 50.0)),
        "markerThickness": max(0.0, min(markerThickness, 1.0)),
        "markerRotation": max(0.0, min(markerRotation, 359.0)),
        "markerColor": markerColor if markerColor in ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"] else "white",
        "letterThickness": max(0.0, min(letterThickness, 50.0)),
        "tdpcCategory": tdpcCategory,
        "tdpcType": tdpcType if tdpcType in ["circle", "cross", "square"] else "circle",
        "tdpcScale": max(0.5, min(tdpcScale, 50.0)),
        "tdpcThickness": max(0.1, min(tdpcThickness, 50.0)),
        "tdpcExtrusion": max(0.0, min(tdpcExtrusion, 1.0)),
        "tdpcRotation": max(0.0, min(tdpcRotation, 359.0)),
        "tdpcColor": tdpcColor if tdpcColor in ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"] else "white",
        "lightColor": lightColor,
        "lightScale": max(-20.0, min(lightScale, 100.0)),
        "lightDistance": max(1.0, min(lightDistance, 50.0)),
        "lightRadius": max(0.1, min(lightRadius, 1.0)),
        "lightHeight": max(0.1, min(lightHeight, 0.25)),
        "safetyAreaType": safetyAreaType if safetyAreaType in ["offset", "multiplier"] else "offset",
        "dValue": max(1.0, min(dValue, 20.0)),
        "multiplier": max(0.5, min(multiplier, 20.0)),
        "offsetDistance": max(0.0, min(offsetDistance, 50.0)),
        "curveAngle": max(0.0, min(curveAngle, 90.0)),
        "netHeight": max(0.0, min(netHeight, 100.0)),
        "safetyNetTransparency": max(0.0, min(safetyNetTransparency, 1.0))
    }

# Bind the tool to the LLM
llm_with_tools = llm.bind_tools([generate_landing_surface_layout])

# Function to get LangChain response with tool call
def get_function_call(prompt: str) -> dict:
    messages = [HumanMessage(content=prompt)]
    response = llm_with_tools.invoke(messages)
    
    # Extract tool call arguments
    for tool_call in response.tool_calls:
        if tool_call["name"] == "generate_landing_surface_layout":
            return tool_call["args"]
    raise ValueError("No valid tool call found in response")

# Function to merge update with previous configuration
def merge_with_previous(new_args, prev_config, prompt):
    if prev_config is None or "update" not in prompt.lower():
        return new_args
    merged_args = prev_config.copy()
    merged_args.update(new_args)
    return generate_landing_surface_layout(**merged_args)

# Main loop
def main():
    while True:
        try:
            user_input = input("Describe the TLOF (Landing Surface) in natural language:\n")
            
            # Step 1: Get user-intended parameters
            user_args = get_function_call(user_input)

            # Step 2: Merge with previous configuration if it's an update
            last_config = get_last_config()
            user_args = merge_with_previous(user_args, last_config, user_input)

            # Step 3: Save to history (store full configuration)
            history_entry = {
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "input": user_input,
                "output": user_args
            }
            save_history(history_entry)

            # Step 4: Print full configuration
            print(json.dumps(user_args, indent=2))

        except KeyboardInterrupt:
            print("\nExiting program.")
            break
        except Exception as e:
            print(f"Error: {e}")
            continue

if __name__ == "__main__":
    main()

Describe the TLOF (Landing Surface) in natural language:
 set light height to 0.20


Error: No valid tool call found in response


Describe the TLOF (Landing Surface) in natural language:
 Generate a rectangular TLOF for a tiltrotor aircraft with 30m x 40m dimensions, elevation 5m, rotation 15 degrees, and 0.6 transparency. Location is [139.6917, 35.6895]. Add a 'V' landing marker in blue, scaled to 8, rotated to 90 degrees.


{
  "aircraft": "tiltrotor",
  "diameter": null,
  "shape_type": "Rectangle",
  "position": [
    139.6917,
    35.6895
  ],
  "sides": null,
  "width": 30,
  "length": 40,
  "height": 0.01,
  "rotation": 15,
  "transparency": 0.6,
  "elevation": 5,
  "markingType": "solid",
  "markingColor": "white",
  "markingThickness": 0.5,
  "dashLength": 1,
  "dashDistance": 1,
  "landingMarker": "V",
  "markerScale": 8,
  "markerThickness": 0.02,
  "markerRotation": 90,
  "markerColor": "blue",
  "letterThickness": 0.5,
  "tdpcCategory": false,
  "tdpcType": "circle",
  "tdpcScale": 5,
  "tdpcThickness": 0.5,
  "tdpcExtrusion": 0.02,
  "tdpcRotation": 0,
  "tdpcColor": "white",
  "lightColor": null,
  "lightScale": 1,
  "lightDistance": 1,
  "lightRadius": 0.3,
  "lightHeight": 0.2,
  "safetyAreaType": "offset",
  "dValue": 10,
  "multiplier": 1.5,
  "offsetDistance": 3,
  "curveAngle": 45,
  "netHeight": 15,
  "safetyNetTransparency": 0.5
}


Describe the TLOF (Landing Surface) in natural language:
 SET marker Color TO RED


{
  "markerColor": "red"
}


Describe the TLOF (Landing Surface) in natural language:
 SET curve Angle 40


{
  "curveAngle": 40
}
